In [ ]:
import sys
import json
import yaml
import pickle
from pathlib import Path

import qlib
import pandas as pd
from qlib.config import REG_CN
from qlib.utils import exists_qlib_data

In [ ]:
CUR_DIR = Path.cwd()
MARKET = "csi300"

In [ ]:
# use default data
# NOTE: need to download data from remote: python scripts/get_data.py qlib_data --target_dir ~/.qlib/qlib_data/cn_data
provider_uri = "~/.qlib/qlib_data/cn_data"  # target_dir
if not exists_qlib_data(provider_uri):
    print(f"Qlib data is not found in {provider_uri}")
    sys.path.append(str(CUR_DIR.parent.parent.joinpath("scripts")))
    from get_data import GetData
    GetData().qlib_data_cn(provider_uri)
qlib.init(provider_uri=provider_uri, region=REG_CN)

In [ ]:
with CUR_DIR.joinpath('estimator_config.yaml').open() as fp:
    estimator_name = yaml.load(fp, Loader=yaml.FullLoader)['experiment']['name']
with CUR_DIR.joinpath(estimator_name, 'exp_info.json').open() as fp:
    latest_id = json.load(fp)['id']
    
estimator_dir = CUR_DIR.joinpath(estimator_name, 'sacred', latest_id)

# read estimator result

In [ ]:
pred_df = pd.read_pickle(estimator_dir.joinpath('pred.pkl'))
report_normal_df = pd.read_pickle(estimator_dir.joinpath('report_normal.pkl'))
report_normal_df.index.names = ['index']

analysis_df = pd.read_pickle(estimator_dir.joinpath('analysis.pkl'))
positions = pickle.load(estimator_dir.joinpath('positions.pkl').open('rb'))

# get label data from qlib

In [ ]:
from qlib.data import D
pred_df_dates = pred_df.index.get_level_values(level='datetime')
features_df = D.features(D.instruments(MARKET), ['Ref($close, -1)/$close - 1'], pred_df_dates.min(), pred_df_dates.max())
features_df.columns = ['label']

# analyze graphs

In [ ]:
from qlib.contrib.report import analysis_model, analysis_position

## analysis position

### report

In [ ]:
analysis_position.report_graph(report_normal_df)

### score IC

In [ ]:
pred_label = pd.concat([features_df, pred_df], axis=1, sort=True).reindex(features_df.index)
analysis_position.score_ic_graph(pred_label)

### cumulative return

In [ ]:
analysis_position.cumulative_return_graph(positions, report_normal_df, features_df)

### risk analysis

In [ ]:
analysis_position.risk_analysis_graph(analysis_df, report_normal_df)

### rank label

In [ ]:
analysis_position.rank_label_graph(positions, features_df, pred_df_dates.min(), pred_df_dates.max())

## analysis model

### model performance

In [ ]:
analysis_model.model_performance_graph(pred_label)